In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('processed_data.csv')
# cor_matrix = df.corr()

In [9]:
import pickle
# f = open("cor_matrix.pkl", 'wb')
# pickle.dump(cor_matrix, f)
cor_matrix = pickle.load("cor_matrix.pkl")

In [63]:
# 根据相关系数选特征
data_selected = []
for i in list(cor_matrix.columns.values):
    if abs(cor_matrix[i]['bad_good'])>=0.05:
        data_selected.append(i)

In [64]:
data_great=df.loc[:, data_selected]
data_great.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285285 entries, 0 to 285284
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   bad_good                   285285 non-null  float64
 1   DEP_SA_OPEN_TENURE_DAYS    285285 non-null  float64
 2   DEP_SA_LAST_TENURE_DAYS    285285 non-null  float64
 3   DEP_SA_AVG_TENURE_DAYS     285285 non-null  float64
 4   DEP_SA_SUMACCOUNT_CNT      285285 non-null  float64
 5   L3_DEP_SA_AVG_ACCOUNT_CNT  285285 non-null  float64
 6   L6_DEP_SA_AVG_ACCOUNT_CNT  285285 non-null  float64
 7   DEP_SA_ACCOUNT_CNT         285285 non-null  float64
 8   LOAN_FLAG                  285285 non-null  float64
dtypes: float64(9)
memory usage: 19.6 MB


In [66]:
outlier_feature=list(data_great.columns)
outlier_feature.remove('bad_good')
for i in outlier_feature:
    mean=data_great[i].mean()
    std=data_great[i].std()
    upper_limit=mean+3*std
    lower_limit=mean-3*std
    condition=(data_great[i]>upper_limit)|(data_great[i]<lower_limit)
    print('outlier numbers:%.2f'%len(data_great.loc[condition]))
    print(i+' outlier number rate:%.6f'%(len(data_great.loc[condition])/len(data_great)))
    print('-'*40)

outlier numbers:1790.00
DEP_SA_OPEN_TENURE_DAYS outlier number rate:0.006274
----------------------------------------
outlier numbers:3332.00
DEP_SA_LAST_TENURE_DAYS outlier number rate:0.011680
----------------------------------------
outlier numbers:2333.00
DEP_SA_AVG_TENURE_DAYS outlier number rate:0.008178
----------------------------------------
outlier numbers:2428.00
DEP_SA_SUMACCOUNT_CNT outlier number rate:0.008511
----------------------------------------
outlier numbers:2386.00
L3_DEP_SA_AVG_ACCOUNT_CNT outlier number rate:0.008364
----------------------------------------
outlier numbers:2355.00
L6_DEP_SA_AVG_ACCOUNT_CNT outlier number rate:0.008255
----------------------------------------
outlier numbers:2428.00
DEP_SA_ACCOUNT_CNT outlier number rate:0.008511
----------------------------------------
outlier numbers:13651.00
LOAN_FLAG outlier number rate:0.047850
----------------------------------------


In [71]:
#根据特征处理异常值
data_filtered = data_great.copy()
for i in data_selected:
    if(i == 'bad_good' or i == 'LOAN_FLAG'):
        continue
    mean=data_filtered[i].mean()
    std=data_filtered[i].std()
    upper_limit=mean+3*std
    lower_limit=mean-3*std
    condition=(data_filtered[i]<=upper_limit)&(data_filtered[i]>=lower_limit)
    data_filtered=data_filtered.loc[condition].copy()
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264607 entries, 3 to 285284
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   bad_good                   264607 non-null  float64
 1   DEP_SA_OPEN_TENURE_DAYS    264607 non-null  float64
 2   DEP_SA_LAST_TENURE_DAYS    264607 non-null  float64
 3   DEP_SA_AVG_TENURE_DAYS     264607 non-null  float64
 4   DEP_SA_SUMACCOUNT_CNT      264607 non-null  float64
 5   L3_DEP_SA_AVG_ACCOUNT_CNT  264607 non-null  float64
 6   L6_DEP_SA_AVG_ACCOUNT_CNT  264607 non-null  float64
 7   DEP_SA_ACCOUNT_CNT         264607 non-null  float64
 8   LOAN_FLAG                  264607 non-null  float64
dtypes: float64(9)
memory usage: 20.2 MB


In [72]:
# 重新归一化
data_filtered = (data_filtered-data_filtered.min())/(data_filtered.max()-data_filtered.min())

In [74]:
f = open('final_data.pkl', 'wb')
pickle.dump(data_filtered, f)

In [69]:
data_great.corr()['bad_good'].abs().sort_values(ascending=False)

bad_good                     1.000000
LOAN_FLAG                    0.588719
DEP_SA_OPEN_TENURE_DAYS      0.163460
DEP_SA_AVG_TENURE_DAYS       0.156396
DEP_SA_LAST_TENURE_DAYS      0.132562
L6_DEP_SA_AVG_ACCOUNT_CNT    0.050658
L3_DEP_SA_AVG_ACCOUNT_CNT    0.050598
DEP_SA_SUMACCOUNT_CNT        0.050487
DEP_SA_ACCOUNT_CNT           0.050487
Name: bad_good, dtype: float64